In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.LoadupSamples import LoadupSamples
from src.predictionModule.FilterSamples import FilterSamples

import numpy as np
import polars as pl
import datetime
import random
import matplotlib.pyplot as plt

import logging
formatted_date = datetime.datetime.now().strftime("%d%b%y_%H%M").lower()

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]

#Output File handler
file_handler = logging.FileHandler(f"notebook-optuna-{formatted_date}.log", mode="w")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Usage
logger.info("This will print to the notebook's output cell")

2025-07-21 10:55:38,850 - This will print to the notebook's output cell


In [2]:
params = {
    "idxAfterPrediction": 5,
    'timesteps': 60,
    'target_option': 'last',
    
    "TreeTime_lstm_units": 64,
    "TreeTime_lstm_num_layers": 3,
    "TreeTime_lstm_dropout": 0.00001,
    "TreeTime_lstm_recurrent_dropout": 0.00001,
    "TreeTime_lstm_learning_rate": 0.001,
    "TreeTime_lstm_optimizer": "adam",
    "TreeTime_lstm_bidirectional": True,
    "TreeTime_lstm_batch_size": 2**14,
    "TreeTime_lstm_epochs": 4,
    "TreeTime_lstm_l1": 0.00001,
    "TreeTime_lstm_l2": 0.00001,
    "TreeTime_inter_dropout": 0.00001,
    "TreeTime_input_gaussian_noise": 0.00001,
    "TreeTime_lstm_conv1d": True,
    "TreeTime_lstm_conv1d_kernel_size": 3,
    "TreeTime_lstm_loss": "mse",
}

In [3]:
stock_group = "group_snp500_finanTo2011"

eval_date = datetime.date(2025,  2,  1)

start_Date = datetime.date(year=2014, month=1, day=1)

ls = LoadupSamples(
    train_start_date=start_Date,
    test_dates=[eval_date],
    group=stock_group,
    params=params,
)

In [4]:
ls.load_samples(main_path="../src/featureAlchemy/bin/")

2025-07-21 10:57:04,402 - Test date 2025-02-01 not found in the database. Resetting to last trading day.


In [5]:
dates = ls.meta_pl_train.select(pl.col("date")).to_series()
Xtree_all = ls.train_Xtree
ytree_all = ls.train_ytree
        
def est_samples(X_all, y_all, dates: pl.Series, end_train_date, test_days):
    train_start = min(dates)  # Start from the earliest date in the dataset
    train_end   = end_train_date

    test_start  = end_train_date + datetime.timedelta(days=1)
    test_end    = end_train_date + datetime.timedelta(days=test_days)

    # slice train
    train_mask = (
        (dates >= train_start) &
        (dates <= train_end)
    ).fill_null(False).to_numpy()

    # slice test
    test_mask = (
        (dates >= test_start) &
        (dates <= test_end)
    ).fill_null(False).to_numpy()

    return (
        X_all[train_mask],
        X_all[test_mask],
        y_all[train_mask],
        y_all[test_mask],
        dates.filter(train_mask),
        dates.filter(test_mask)
    )

In [ ]:
import optuna
logger.setLevel(logging.INFO)
def objective(trial: optuna.Trial) -> float:
    # Suggest hyperparameters
    opt_params = {
        "FilterSamples_lincomb_q_up": 0.95,
        "FilterSamples_lincomb_lr": trial.suggest_float("lincomb_lr", 5e-5, 1e-2, log=True),
        "FilterSamples_lincomb_epochs": trial.suggest_int("lincomb_epochs", 300, 1000, step = 100),
        "FilterSamples_lincomb_probs_noise_std": trial.suggest_float("lincomb_noise_std", 0.005, 0.2, log=True),
        "FilterSamples_lincomb_subsample_ratio": trial.suggest_float("lincomb_subsample_ratio", 0.05, 0.5),
        "FilterSamples_lincomb_sharpness": trial.suggest_float("lincomb_sharpness", 0.2, 2.5, log=True),
        "FilterSamples_lincomb_featureratio": trial.suggest_float("lincomb_featureratio", 0.05, 0.99, log=True),
        "FilterSamples_lincomb_itermax": trial.suggest_int("lincomb_itermax", 1, 2),
        "FilterSamples_lincomb_show_progress": False,
    }
    
    end_train_dates = sorted([
        datetime.date(2024,  7,  1) - datetime.timedelta(days=i*120 + random.randint(-10,10)) 
        for i in range(10)
    ])
    
    train_days = trial.suggest_int("num_train_days", 150, 600, step=30)  # 6 months to 2 years of training data
    test_days = trial.suggest_int("num_test_days", 7, 14)  

    # Build and evaluate FilterSamples for each instance
    scores_train = []
    scores_test = []
    for i, end_date in enumerate(end_train_dates):
        try:
            (
                Xtree_train,
                Xtree_test,
                ytree_train,
                ytree_test,
                samples_dates_train,
                samples_dates_test
            ) = est_samples(Xtree_all, ytree_all, dates, end_date, test_days=test_days)

            opt_params["FilterSamples_days_to_train_end"] = train_days
            fs = FilterSamples(
                Xtree_train = Xtree_train,
                ytree_train = ytree_train,
                treenames = ls.featureTreeNames,
                Xtree_test = Xtree_test,
                samples_dates_train = samples_dates_train,
                samples_dates_test = samples_dates_test,
                ytree_test = ytree_test,
                params = opt_params
            )
            _, _, s_tr, s_te = fs.run()
            s_tr = 1.0 if s_tr is None or np.isnan(s_tr) or s_tr<0.5 else s_tr
            s_te = 1.0 if s_te is None or np.isnan(s_te) or s_te<0.5 else s_te
            
        except ValueError:
            s_tr = s_te = 1.0
            
        scores_train.append(s_tr)
        scores_test.append(s_te)    
        
    logger.info(f"Trial number {trial.number}")
    logger.info(f"All scores (train) = {scores_train}")
    logger.info(f"All scores (test) = {scores_test}") 
    
    fin_score = np.mean(np.log(scores_test))
    logger.info(f"Log mean of scores (train) {np.mean(np.log(scores_train))}")
    logger.info(f"Log mean of scores (test) {np.mean(np.log(scores_test))}")
    
    return float(fin_score)

optuna.logging.enable_propagation()
study = optuna.create_study(
    direction="maximize",
)
study.optimize(objective, timeout=60*60*1)

logger.info(f"Best parameters: {study.best_params}")
logger.info(f"Best score: {study.best_value}")

df = study.trials_dataframe()
logger.info("\nTrials DataFrame:")
logger.info(df.sort_values("value").to_string())

param_importances = optuna.importance.get_param_importances(study)
logger.info("Parameter Importances:")
for key, value in param_importances.items():
    logger.info(f"{key}: {value}")

c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-07-21 10:57:38,924] A new study created in memory with name: no-name-61b32e32-1888-4a61-a932-188cfd76d8e1


2025-07-21 10:57:38,924 - A new study created in memory with name: no-name-61b32e32-1888-4a61-a932-188cfd76d8e1
2025-07-21 10:57:43,487 - FilterSamples: (train) mean of y values 1.0029710193986436.
2025-07-21 10:57:43,492 - FilterSamples: (test) mean of y values 1.025278947686991.
2025-07-21 10:57:45,142 - FilterSamples: Best init score 1.0130
2025-07-21 10:57:56,209 -   Mean target (train): 1.0280265161297693
2025-07-21 10:57:56,210 -   Max distance between idces: 239
2025-07-21 10:57:56,225 -   Mean target (test): 0.9460121135864914
2025-07-21 10:57:56,227 -   Fraction of days with no coverage by test mask: 0.00%
2025-07-21 10:57:56,901 - FilterSamples: Best init score 1.0130
2025-07-21 10:57:59,790 -   Mean target (train): 1.0373477993101443
2025-07-21 10:57:59,790 -   Max distance between idces: 3457
2025-07-21 10:57:59,810 -   Mean target (test): 0.0
2025-07-21 10:57:59,811 -   Fraction of days with no coverage by test mask: 100.00%
2025-07-21 10:58:00,329 - FilterSamples: Final s

[I 2025-07-21 11:00:50,120] Trial 0 finished with value: 0.032456209081062955 and parameters: {'lincomb_lr': 0.0020804864413479884, 'lincomb_epochs': 500, 'lincomb_noise_std': 0.00869419509739274, 'lincomb_subsample_ratio': 0.09833144605437866, 'lincomb_sharpness': 2.1676595580946154, 'lincomb_featureratio': 0.3118010846671437, 'lincomb_itermax': 2, 'num_train_days': 600, 'num_test_days': 7}. Best is trial 0 with value: 0.032456209081062955.


2025-07-21 11:00:50,120 - Trial 0 finished with value: 0.032456209081062955 and parameters: {'lincomb_lr': 0.0020804864413479884, 'lincomb_epochs': 500, 'lincomb_noise_std': 0.00869419509739274, 'lincomb_subsample_ratio': 0.09833144605437866, 'lincomb_sharpness': 2.1676595580946154, 'lincomb_featureratio': 0.3118010846671437, 'lincomb_itermax': 2, 'num_train_days': 600, 'num_test_days': 7}. Best is trial 0 with value: 0.032456209081062955.
2025-07-21 11:00:54,343 - FilterSamples: (train) mean of y values 1.0029848242955464.
2025-07-21 11:00:54,347 - FilterSamples: (test) mean of y values 1.017911909879729.
2025-07-21 11:00:55,621 - FilterSamples: Best init score 1.0133
2025-07-21 11:01:00,351 -   Mean target (train): 1.0681164046340987
2025-07-21 11:01:00,351 -   Max distance between idces: 2751
2025-07-21 11:01:00,370 -   Mean target (test): 1.000923711380093
2025-07-21 11:01:00,371 -   Fraction of days with no coverage by test mask: 33.33%
2025-07-21 11:01:00,836 - FilterSamples: Bes

[I 2025-07-21 11:04:00,450] Trial 1 finished with value: 0.007644491918385907 and parameters: {'lincomb_lr': 0.003147039898909169, 'lincomb_epochs': 700, 'lincomb_noise_std': 0.0057623807103516856, 'lincomb_subsample_ratio': 0.28467879869386203, 'lincomb_sharpness': 1.6883671612457338, 'lincomb_featureratio': 0.08739525811676423, 'lincomb_itermax': 2, 'num_train_days': 570, 'num_test_days': 13}. Best is trial 0 with value: 0.032456209081062955.


...

2025-07-21 14:50:28,814 - Trial 83 finished with value: 0.015560625448858622 and parameters: {'lincomb_lr': 0.005012499717503836, 'lincomb_epochs': 900, 'lincomb_noise_std': 0.005439417978535376, 'lincomb_subsample_ratio': 0.07142663585048824, 'lincomb_sharpness': 0.9547700684533834, 'lincomb_featureratio': 0.8668291568444948, 'lincomb_itermax': 2, 'num_train_days': 570, 'num_test_days': 7}. Best is trial 34 with value: 0.035285781933109536.
2025-07-21 14:50:32,712 - FilterSamples: (train) mean of y values 1.0029713184105533.
2025-07-21 14:50:32,712 - FilterSamples: (test) mean of y values 1.0224460859678643.
2025-07-21 14:50:33,930 - FilterSamples: Best init score 1.0095
2025-07-21 14:50:39,650 -   Mean target (train): 1.0130258513887258
2025-07-21 14:50:39,650 -   Max distance between idces: 458
2025-07-21 14:50:39,670 -   Mean target (test): 0.9824298435890302
2025-07-21 14:50:39,672 -   Fraction of days with no coverage by test mask: 0.00%
2025-07-21 14:50:40,489 - FilterSamples: B

[I 2025-07-21 14:54:13,822] Trial 84 finished with value: 0.01643837719376044 and parameters: {'lincomb_lr': 0.0002685960306591099, 'lincomb_epochs': 1000, 'lincomb_noise_std': 0.005032100197017448, 'lincomb_subsample_ratio': 0.08400885688690854, 'lincomb_sharpness': 0.6565516659996897, 'lincomb_featureratio': 0.62469123416243, 'lincomb_itermax': 2, 'num_train_days': 540, 'num_test_days': 11}. Best is trial 34 with value: 0.035285781933109536.


2025-07-21 14:54:13,822 - Trial 84 finished with value: 0.01643837719376044 and parameters: {'lincomb_lr': 0.0002685960306591099, 'lincomb_epochs': 1000, 'lincomb_noise_std': 0.005032100197017448, 'lincomb_subsample_ratio': 0.08400885688690854, 'lincomb_sharpness': 0.6565516659996897, 'lincomb_featureratio': 0.62469123416243, 'lincomb_itermax': 2, 'num_train_days': 540, 'num_test_days': 11}. Best is trial 34 with value: 0.035285781933109536.
2025-07-21 14:54:17,772 - FilterSamples: (train) mean of y values 1.0029791091962712.
2025-07-21 14:54:17,777 - FilterSamples: (test) mean of y values 0.9919151448566145.
2025-07-21 14:54:18,831 - FilterSamples: Best init score 1.0165
2025-07-21 14:54:24,342 -   Mean target (train): 1.0306152343320576
2025-07-21 14:54:24,346 -   Max distance between idces: 6692
2025-07-21 14:54:24,350 -   Mean target (test): 0.0
2025-07-21 14:54:24,357 -   Fraction of days with no coverage by test mask: 100.00%
2025-07-21 14:54:25,174 - FilterSamples: Best init sco

[I 2025-07-21 14:57:58,192] Trial 85 finished with value: 0.008608380619492078 and parameters: {'lincomb_lr': 0.0005144372649839978, 'lincomb_epochs': 1000, 'lincomb_noise_std': 0.01664421802943008, 'lincomb_subsample_ratio': 0.18345290064460817, 'lincomb_sharpness': 1.7841542744431198, 'lincomb_featureratio': 0.8132195939810432, 'lincomb_itermax': 2, 'num_train_days': 450, 'num_test_days': 8}. Best is trial 34 with value: 0.035285781933109536.


2025-07-21 14:57:58,192 - Trial 85 finished with value: 0.008608380619492078 and parameters: {'lincomb_lr': 0.0005144372649839978, 'lincomb_epochs': 1000, 'lincomb_noise_std': 0.01664421802943008, 'lincomb_subsample_ratio': 0.18345290064460817, 'lincomb_sharpness': 1.7841542744431198, 'lincomb_featureratio': 0.8132195939810432, 'lincomb_itermax': 2, 'num_train_days': 450, 'num_test_days': 8}. Best is trial 34 with value: 0.035285781933109536.
2025-07-21 14:57:58,199 - Best parameters: {'lincomb_lr': 0.0005987115021101787, 'lincomb_epochs': 700, 'lincomb_noise_std': 0.006052949214035855, 'lincomb_subsample_ratio': 0.1988760097048758, 'lincomb_sharpness': 0.27575438222291876, 'lincomb_featureratio': 0.6489784214080768, 'lincomb_itermax': 1, 'num_train_days': 450, 'num_test_days': 8}
2025-07-21 14:57:58,200 - Best score: 0.035285781933109536
2025-07-21 14:57:58,211 - 
Trials DataFrame:
2025-07-21 14:57:58,231 -     number     value             datetime_start          datetime_complete    

In [7]:
df.to_parquet(f"notebook-optuna-{formatted_date}.parquet", index=False)